In [1]:
# Import standard library modules
import sys

# Set the relative path to the project root directory
relative_path_to_root = "../../../"

# Add the project root to the system path for importing in-house modules
sys.path.append(relative_path_to_root)

# Import in-house modules from the 'utilities' package
from utilities import calculate_bollinger_bands, calculate_rsi, calculate_daily_volatility
from utilities import print_title, print_label, print_footer
from utilities import generate_trading_signals
from utilities import fetch_and_download_sp500_data, sp500_data_for_today
from utilities import print_dataframe_report, save_data, load_data

In [2]:
# Import libraries for data analysis and visualization
import pandas as pd

# Import libraries for signal processing and peak detection
from scipy.signal import find_peaks
import numpy as np

# Import libraries for plotting and visualization
import matplotlib.pyplot as plt
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

# Import libraries for data storage and retrieval
from pathlib import Path

In [3]:
def calculate_rsi(historical_prices, window=14):
    # Ensure calculations are performed on the 'Adj Close' column
    delta = historical_prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [4]:
file_name = "sp500_adj_close_raw"
file_path = f"../../../data/raw_data/{file_name}"
loaded_data = load_data(file_path)


╔═══════════════════════════════════════════════════════════════╗
║File `sp500_adj_close_raw.csv` loaded from `sp500_adj_close_raw.zip`║
╚═══════════════════════════════════════════════════════════════╝


In [5]:
loaded_data

,Date,Ticker,Adjusted Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
0,2023-10-24 00:00:00+00:00,A,104.928497,-0.021204,0.409554,0.000000,121.677983,128.300623,133.159750,115.267511,98.748274,100.488594,138.878311,buy
1,2023-10-24 00:00:00+00:00,AAPL,172.557022,-0.013492,0.188448,0.000000,186.475520,184.511592,189.937574,194.716175,162.902431,166.040359,197.361069,short
2,2023-10-24 00:00:00+00:00,ABBV,141.150238,-0.007177,0.208797,0.000000,140.884356,154.118350,160.623433,141.142256,130.635856,132.747421,155.765976,short
3,2023-10-24 00:00:00+00:00,ABNB,122.010002,-0.028768,0.456553,0.000000,130.258200,140.167450,145.392425,130.042030,111.038970,114.089996,147.500000,short
4,2023-10-24 00:00:00+00:00,ABT,92.954979,-0.013079,0.170163,0.000000,100.178232,106.883076,106.050604,98.488930,89.118162,91.033325,108.808609,buy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124104,2024-10-21 15:59:00-04:00,XYL,132.570007,-0.013029,0.148419,44.305416,133.478516,134.724480,130.457142,137.626554,131.833447,126.709999,137.529999,short
124105,2024-10-21 15:59:00-04:00,YUM,134.039993,-0.001713,0.194556,25.791286,134.851627,133.612766,133.874638,140.514409,130.791594,129.710007,139.919998,sell
124106,2024-10-21 15:59:00-04:00,ZBH,105.209999,-0.010347,0.172217,42.876266,107.737997,108.358195,115.717960,108.973089,101.459495,101.769997,115.912369,sell
124107,2024-10-21 15:59:00-04:00,ZBRA,372.390015,-0.008150,0.159147,59.331050,353.222800,334.661000,308.857250,379.090798,361.437199,320.339996,377.679993,sell
